In [ ]:
# EDA enligt https://www.kaggle.com/code/ayushikaushik/eda-regression-analysis#Preprocessing-the-data

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

train_data = pd.read_csv('data/stores_train.csv')

print(f"Shape of training data: {train_data.shape}\nFeatures available: {train_data.columns}")

In [ ]:
# To see all the columns in output this can be done.
pd.options.display.max_columns=None
# To see all rows change max_columns with max_rows

train_data.head()

In [ ]:
# Check if there are missing values => true, so there are
train_data.isnull().sum().any()

In [ ]:
# Replace NaN in mall_name column with 'No mall'
train_data.mall_name = train_data.mall_name.fillna('No mall')
train_data.address = train_data.address.fillna('No address')
train_data.chain_name = train_data.chain_name.fillna('No chain')

In [ ]:
train_data.head()

In [ ]:
# Check if there are missing values => false, so there aren't anymore :)
train_data.isnull().sum().any()

In [ ]:
# 'year' column is redundant, remove it
train_data = train_data.drop('year',axis=1)

In [ ]:
train_data.head()

In [ ]:
#train_data['store_name'] = pd.factorize(train_data['store_name'])[0]

In [ ]:
#train_data.head()

In [ ]:
sns.distplot(train_data['revenue'],hist=False)
plt.title('Distribution of Target variable')
sns.despine() # removes top and right border from the figure

In [ ]:
sns.distplot(np.log(train_data['revenue']),hist=False)
plt.title('Distribution of Target variable')

In [ ]:
# The data is ≈ normally distributed when plotted in log

In [ ]:
# Make new columns for less specified plaace hierarchy to group data together

In [ ]:
sns.countplot(x=train_data["plaace_hierarchy_id"])

In [ ]:
# % In[8]